In [60]:
import os
import subprocess


import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import gridspec
from sklearn.linear_model import LinearRegression as LR

import pandas as pd
from pingouin import mixed_anova, anova, pairwise_tukey, pairwise_tests, rm_anova
from statsmodels.regression.mixed_linear_model import MixedLM

import TwoPUtils as tpu
import STX3KO_analyses as stx
from STX3KO_analyses import utilities as u

ko_mice = stx.ymaze_sess_deets.ko_mice
ctrl_mice = stx.ymaze_sess_deets.ctrl_mice

plt.rcParams['pdf.fonttype']=42
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
sess = u.load_single_day('4467975.5',0)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 0}


In [62]:
print(sess.iscell.shape)
print(sess.iscell.sum(axis=0))
sess.trial_matrices['spks'].shape

(2259, 2)
[1578.         1998.22651012]


(55, 30, 1578)

In [63]:
mouse = '4467975.5'
sessions = stx.ymaze_sess_deets.KO_sessions[mouse]
base_oak_dir = '/home/mplitt/roi_checks'
base_local_dir = '/home/mplitt/YMazeSessPkls_HandCurated'
for sess in sessions[:1]:
    print(sess)
    csv_oak_path = f"{base_oak_dir}/{mouse}/{sess['date']}/{sess['scene']}/{sess['scene']}_{sess['session']:>03}_{sess['scan']:>03}/suite2p/plane0/roi_check.csv"
    csv_local_path = f"{base_local_dir}/{mouse}/{sess['date']}/{sess['scene']}_{sess['session']}_roi_check.csv"
    print(csv_local_path)
    subprocess.run(['cp', csv_oak_path,  csv_local_path])

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 0}
/home/mplitt/YMazeSessPkls_HandCurated/4467975.5/28_09_2020/YMaze_LNovel_1_roi_check.csv


In [64]:
for i, sess_deets in enumerate(sessions[:1]):
    sess = u.load_single_day(mouse,i)

    csv_local_path = f"{base_local_dir}/{mouse}/{sess_deets['date']}/{sess_deets['scene']}_{sess_deets['session']}_roi_check.csv"
    roi_check = pd.read_csv(csv_local_path)

    roi_check_filt = roi_check.loc[ sess.iscell[:,0]>0,:]
    red_mask = roi_check_filt['Cre']>0

    for k,v in sess.timeseries.items():
        if v.shape[0]>1:
            sess.timeseries[k] = v[red_mask,:]

    for k,v in sess.trial_matrices.items():
        if len(v.shape)==3:
            sess.trial_matrices[k] = v[:,:, red_mask]

    for lr in sess.place_cell_info.keys():
        for k, v in sess.place_cell_info[lr].items():
            sess.place_cell_info[lr][k] = v[red_mask]
        

    tpu.sess.save_session(sess,'/home/mplitt/YMazeSessPkls_HandCurated')

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 0}


In [53]:
print(sess.iscell[:,0].sum(), red_mask.sum(), sess.timeseries['spks'].shape)

894.0 542 (542, 38161)


In [56]:
sess.place_cell_info.keys()

dict_keys(['left', 'right'])

In [36]:
roi_check_filt = roi_check.loc[ sess.iscell[:,0]>0,:]

In [37]:
roi_check_filt.shape

(973, 5)

In [38]:
sess.timeseries['spks'].shape

(973, 35052)

In [39]:
red_mask = roi_check_filt['Cre']>0

In [40]:
red_mask.sum()

480

In [43]:
for k,v in sess.timeseries.items():
    print(k,v.shape)

F (973, 35052)
Fneu (973, 35052)
spks (973, 35052)
F_dff (973, 35052)
F_dff_norm (973, 35052)
spks_norm (973, 35052)
licks (1, 35052)
speed (1, 35052)
spks_norm_fast (973, 35052)
t (1, 35052)
spks_th (973, 35052)
LR (1, 35052)
reward (1, 35052)
block_number (1, 35052)
spks_nostop (973, 35052)


In [46]:
for k,v in sess.trial_matrices.items():
    print(k,len(v.shape))

F_dff 3
bin_edges 1
bin_centers 1
spks 3
F_dff_norm 3
spks_norm 3
licks 2
speed 2
spks_norm_fast 3
t 2
spks_th 3
spks_nostop 3
